In [4]:
import requests
import PyPDF2 
import os
from bs4 import BeautifulSoup
import numpy as np
import urllib.parse
import re
import pandas as pd
import spacy
import difflib
import joblib

In [21]:
links =[
    'https://www.millenniumtfi.pl/dokumenty/id/6.html',
    'https://www.aliortfi.com/dokumenty,karta-informacyjna.1.html',
    'https://www.tfi.bnpparibas.pl/dokumenty,kluczowe-informacje-dla-inwestorow.html',
    'https://www.nntfi.pl/informacje-i-dokumenty/dokumenty-funduszy/kluczowe-informacje-dla-inwestorow',
    'https://noblefunds.pl/dokumenty-kiid',
    'https://www.pkotfi.pl/dokumenty-do-pobrania/kiid/',
    'https://rockbridge.pl/do-pobrania#kiid',
    'https://www.superfund.pl/dokumenty?type=kiid'
    'https://www.uniqa.pl/emerytura-i-inwestycje/dokumenty/uniqa-sfio/',
    'https://www.vigcq-tfi.pl/dokumenty.html'
]

In [2]:
def download_pdf(hrefs, start_id=0):
    
    n_files = 0
    
    for href in hrefs:
        n_files += 1
        
        response = requests.get(href)
  
        pdf = open(os.path.join("data", os.path.basename(href)), 'wb')
        pdf.write(response.content)
        pdf.close()
        print("File", os.path.basename(href), "downloaded")

In [13]:
def pdf_2_text(pdf):
    return_str = False
    if type(pdf) == str:
        pdf = [pdf]
        return_str = True
    
    content = dict()
    
    for fname in pdf:
        
        print(f"Reading {fname}...")
        
        content[fname] = ''
        
        pdfFileObj = open(os.path.join("data",fname), 'rb') 
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj) 
        
        try:
            numPages = pdfReader.numPages
        except:
            pass
        else:
            for j in range(pdfReader.numPages):
                pageObj = pdfReader.getPage(j)
                content[fname] += pageObj.extractText().replace("\n", ' ')
                

        pdfFileObj.close()
            
    return list(content.values())[0] if return_str else content

In [4]:
def get_pdf_hrefs(urls):
    if type(urls) == str:
        urls = [urls]
        
    hrefs = []
    for url in urls:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        links = soup.find_all('a')

        for link in links:
            if ('.pdf' in link.get('href', [])):
                hrefs.append(urllib.parse.urljoin(url, link.get('href')))
    
    hrefs = np.unique(hrefs).tolist()
    return hrefs

In [5]:
def is_kiid(content):
    if not re.search('kluczowe( |\n|$)*informacje( |\n|$)*dla( |\n|$)*inwestorów', content.lower()[:int(len(content)/4)]):
        return False
    return True

def keep_kiids(content):
    c = content.copy()
    for key in content.keys():
        if not is_kiid(c[key]):
            del c[key]
    return c

In [10]:
def META(content,filename='TMP_KIID_META.csv'):   
    pd.DataFrame({
        'ID_KIID': range(1, len(content) + 1),
        'ID_ZESPOLU': (4,) * len(content),
        'NAZWA_PLIKU': content.keys()
    }).to_csv(filename, index=False,encoding="cp1250",sep=";")
    print('Saved to file: {}'.format(filename))

In [10]:
hrefs = get_pdf_hrefs(links)

In [11]:
download_pdf(hrefs)

File 19022019_Noble_Fund_Obligacji_KIID.pdf downloaded
File 20190719_Open_Finance_Obligacji.pdf downloaded
File 20190801_Open_Finance_Konserwatywny.pdf downloaded
File 20200219_Open_Finance_Akcji.pdf downloaded
File 20200219_Open_Finance_Akcji_MIS.pdf downloaded
File 20200219_Open_Finance_Aktywnej_Alokacji.pdf downloaded
File 20200219_Open_Finance_Stabilnego_Wzrostu.pdf downloaded
File Open_Finance_Akcji_20190219.pdf downloaded
File Open_Finance_Akcji_20190719.pdf downloaded
File Open_Finance_Akcji_20200101.pdf downloaded
File Open_Finance_Akcji_MIS_20190219.pdf downloaded
File Open_Finance_Akcji_MIS_20190719.pdf downloaded
File Open_Finance_Akcji_MIS_20200101.pdf downloaded
File Open_Finance_Aktywnej_Alokacji_20190219.pdf downloaded
File Open_Finance_Aktywnej_Alokacji_20190719.pdf downloaded
File Open_Finance_Aktywnej_Alokacji_20200101.pdf downloaded
File Open_Finance_Konserwatywny_20190801.pdf downloaded
File Open_Finance_Obligacji_20190219.pdf downloaded
File Open_Finance_Obligacji_

File 20201009_Noble_Fund_Akcji_Amerykańskich_KIID.pdf downloaded
File Noble_Fund_Africa_and_Frontier_20161202.pdf downloaded
File Noble_Fund_Africa_and_Frontier_20170220.pdf downloaded
File Noble_Fund_Africa_and_Frontier_20170320.pdf downloaded
File Noble_Fund_Africa_and_Frontier_20180219.pdf downloaded
File Noble_Fund_Africa_and_Frontier_KIID.pdf downloaded
File Noble_Fund_Akcji_20161202.pdf downloaded
File Noble_Fund_Akcji_20170220.pdf downloaded
File Noble_Fund_Akcji_20180219.pdf downloaded
File Noble_Fund_Akcji_Amerykańskich_20210101.pdf downloaded
File Noble_Fund_Akcji_Europejskich_20210101.pdf downloaded
File Noble_Fund_Akcji_KIID.pdf downloaded
File Noble_Fund_Akcji_MiSS_20161202.pdf downloaded
File Noble_Fund_Akcji_MiSS_20170220.pdf downloaded
File Noble_Fund_Akcji_MiSS_20170320.pdf downloaded
File Noble_Fund_Akcji_MiSS_20180219.pdf downloaded
File Noble_Fund_Akcji_MiSS_KIID.pdf downloaded
File Noble_Fund_Akcji_MiŚS_20210101.pdf downloaded
File Noble_Fund_Akcji_Polskich_2021010

File 553f7897-a737-48cf-9030-d8c2002fde37.pdf downloaded
File 5b819398-58ab-4226-8266-a4721879cd91.pdf downloaded
File 609e978e-a69f-44a4-a188-8c8de19bc46e.pdf downloaded
File 60b09377-60f2-4f8c-a71b-d7651b95495b.pdf downloaded
File 63afcd17-6e41-417f-92f3-e98d0585f940.pdf downloaded
File 63d85099-457c-4c56-b4e6-11857a929c76.pdf downloaded
File 66988715-b2c9-4051-a7ed-ed056e69f481.pdf downloaded
File 69c4cde0-9c3d-4c26-9c93-7e669e855026.pdf downloaded
File 6d0d4dc5-ff68-4788-9f9c-b635498ae844.pdf downloaded
File 6ee37c98-99b2-4991-96b2-0e6672722ddf.pdf downloaded
File 721c4054-e850-479a-8720-879acaddb880.pdf downloaded
File 74e46569-260b-4db3-87d2-eacbf18ad1d3.pdf downloaded
File 76e7d67c-e9ba-40bd-95e1-f011add148fb.pdf downloaded
File 774de813-ee13-4a58-87a5-a5d3d2145412.pdf downloaded
File 776dbbbc-256b-4ce9-8a10-33db2408888b.pdf downloaded
File 7817fab9-91c7-4534-863e-c3ed148aed9c.pdf downloaded
File 7913ea4c-0456-42f2-885f-2609b1ea3aa4.pdf downloaded
File 7a05af2a-3b0b-4a54-9424-81

File 06.C-QUADRAT%20GreenStars%20ESG%20roczne%20sprawozdanie%20finansowe%202021-sig.pdf downloaded
File 07.VIG%20C-QUADRAT%20SFIO_Owiadczenie%20depozytariusza%202021-sig.pdf downloaded
File 08.C-QUADRAT%20SFIO_informacja%20222b%20i%20222d.pdf downloaded
File 09.VIG%20C-QUADRAT%20SFIO_Sprawozdanie%20z%20dziaalnoci%20AFI_2021.pdf downloaded
File 01.VIG%20C-QUADRAT%20SFIO_sprawozdanie%20z%20badania_2021-sig-sig.pdf downloaded
File 02.C-QUADRAT%20SFIO%20Subfundusz%20ARTS%20Total%20Return%20Bond_sprawozdanie%20z%20badania%202021-sig.pdf downloaded
File 03.C-QUADRAT%20SFIO%20Subfundusz_GreenStars%20ESG_sprawozdanie%20z%20badania%202021-sig.pdf downloaded
File 04.VIG%20C-QUADRAT%20SFIO_poczone%20sprawozdanie%20finansowe%202021-sig.pdf downloaded
File Ocena_stosowania_Zasad___adu_Korporacyjnego_w_2021r_.pdf downloaded
File Realizacja_Polityki_zaanga__owania_w_2021r..pdf downloaded
File Aktywa_i_bilans_sprzeda__y_subfunduszy_29042022.pdf downloaded
File Tabela_Op__at_KupFundusz.pdf downloaded
F

In [14]:
content = pdf_2_text([os.path.basename(href) for href in hrefs])

Reading 19022019_Noble_Fund_Obligacji_KIID.pdf...
Reading 20190719_Open_Finance_Obligacji.pdf...
Reading 20190801_Open_Finance_Konserwatywny.pdf...
Reading 20200219_Open_Finance_Akcji.pdf...
Reading 20200219_Open_Finance_Akcji_MIS.pdf...
Reading 20200219_Open_Finance_Aktywnej_Alokacji.pdf...
Reading 20200219_Open_Finance_Stabilnego_Wzrostu.pdf...
Reading Open_Finance_Akcji_20190219.pdf...
Reading Open_Finance_Akcji_20190719.pdf...
Reading Open_Finance_Akcji_20200101.pdf...
Reading Open_Finance_Akcji_MIS_20190219.pdf...
Reading Open_Finance_Akcji_MIS_20190719.pdf...
Reading Open_Finance_Akcji_MIS_20200101.pdf...
Reading Open_Finance_Aktywnej_Alokacji_20190219.pdf...
Reading Open_Finance_Aktywnej_Alokacji_20190719.pdf...
Reading Open_Finance_Aktywnej_Alokacji_20200101.pdf...
Reading Open_Finance_Konserwatywny_20190801.pdf...
Reading Open_Finance_Obligacji_20190219.pdf...
Reading Open_Finance_Obligacji_20190719.pdf...
Reading Open_Finance_Oszczednosciowy_20190219.pdf...
Reading Open_Finan

Xref table not zero-indexed. ID numbers for objects will be corrected.


Reading 20190219_Noble_Noble_Fund_Akcji_KIID.pdf...
Reading 20190219_Noble_Noble_Fund_Akcji_MiSS_KIID.pdf...
Reading 20190219_Noble_Noble_Fund_Global_Return_KIID.pdf...
Reading 20190219_Noble_Noble_Fund_Mieszany_KIID.pdf...
Reading 20190219_Noble_Noble_Fund_Oszczednosciowy_KIID.pdf...
Reading 20190219_Noble_Noble_Fund_Stabilnego_Wzrostu_Plus_KIID.pdf...
Reading 20190219_Noble_Noble_Fund_Timingowy_KIID.pdf...
Reading 20190524_Noble_Fund_Obligacji_KIID.pdf...


Xref table not zero-indexed. ID numbers for objects will be corrected.
Xref table not zero-indexed. ID numbers for objects will be corrected.


Reading 20190708_Noble_Fund_Mieszany_KIID.pdf...
Reading 20190716_Noble_Fund_Africa_and_Frontier_KIID.pdf...


...

Reading 04.VIG%20C-QUADRAT%20SFIO_poczone%20sprawozdanie%20finansowe%202021-sig.pdf...


Object 43 0 found
Object 73 0 found
Object 96 0 found
Object 99 0 found
Object 100 0 found
Object 106 0 found
Object 101 0 found
Object 102 0 found
Object 105 0 found
Object 107 0 found
Object 110 0 found
Object 111 0 found
Object 114 0 found
Object 115 0 found
Object 118 0 found
Object 119 0 found
Object 125 0 found
Object 120 0 found
Object 121 0 found
Object 124 0 found
Object 129 0 found
Object 128 0 found
Object 162 0 found
Object 168 0 found
Object 163 0 found
Object 164 0 found
Object 167 0 found
Object 169 0 found
Object 171 0 found
Object 172 0 found
Object 178 0 found
Object 173 0 found
Object 174 0 found
Object 177 0 found
Object 179 0 found
Object 186 0 found
Object 180 0 found
Object 181 0 found
Object 185 0 found
Object 187 0 found
Object 189 0 found
Object 190 0 found
Object 242 0 found
Object 244 0 found
Object 245 0 found
Object 252 0 found
Object 246 0 found
Object 247 0 found
Object 251 0 found
Object 253 0 found
Object 255 0 found
Object 256 0 found
Object 299 0 fou

Reading Ocena_stosowania_Zasad___adu_Korporacyjnego_w_2021r_.pdf...
Reading Realizacja_Polityki_zaanga__owania_w_2021r..pdf...
Reading Aktywa_i_bilans_sprzeda__y_subfunduszy_29042022.pdf...
Reading Tabela_Op__at_KupFundusz.pdf...
Reading Og__oszenie_o_aktualizacji_Prospektu_informacyjnego_VIG_C-QUADRAT_SFIO.pdf...
Reading Og__oszenie_o_aktualizacji_Prospektu_informacyjnego_VIG_C-QUADRAT_FIO.pdf...
Reading KIID_VIG02_2022-05-27.pdf...
Reading KIID_VIG02B_2022-05-27.pdf...
Reading KIID_VIG03_2022-05-27.pdf...
Reading KIID_VIG03B_2022-05-27.pdf...
Reading KIID_VIG04_2022-05-27.pdf...
Reading KIID_VIG04B_2022-05-27.pdf...
Reading KIID_VIG05_2022-05-27.pdf...
Reading KIID_VIG05B_2022-05-27.pdf...
Reading KIID_VIG06_2022-05-27.pdf...
Reading KIID_VIG06B_2022-05-27.pdf...
Reading Aktywa_i_bilans_sprzeda__y_subfunduszy_31052022.pdf...
Reading KIID_VIG01B_2022-06-29.pdf...
Reading KIID_VIG01_2022-06-29.pdf...
Reading 20220630_Tabela_Op__at__Phinance.pdf...
Reading KIID_VIG07_2022-07-01.pdf...
R

Xref table not zero-indexed. ID numbers for objects will be corrected.


Reading Tabela_Op__at_Q_Value_01092022.pdf...
Reading Tabela_Op__at_Starfunds_01092022.pdf...
Reading Tabela_Op__at_VIG_TFI_01.09.2022.pdf...
Reading Aktywa_i_bilans_sprzeda__y_subfunduszy_31082022.pdf...
Reading Informacja_Administratora_dla_IKE_IKZE.pdf...
Reading Og__oszenie.pdf...
Reading KIID_VIG08B_2022-09-19.pdf...
Reading KIID_VIG08_2022-09-19.pdf...
Reading Og__oszenie_o_skr__ceniu_subskrypcji_C-QUADRAT_Total_Return_Flexible.pdf...
Reading KIID_VIG08B_2022-10-03.pdf...
Reading KIID_VIG08_2022-10-03.pdf...
Reading Tabela_op__at_za_zarz__dzanie_03102022.pdf...
Reading Rachunki_naby__.pdf...
Reading Tabela_Op__at_VIG_TFI_03102022.pdf...
Reading Tabela_Op__at_VIG_TFI_03102022.pdf...
Reading Tabela_Op__at_VIG_TFI_03102022.pdf...
Reading Aktywa_i_bilans_sprzeda__y_subfunduszy_30092022.pdf...
Reading Ogloszenie_o_blednej_wycenie_aktywow_netto.pdf...
Reading VIG07_30.09.2022_Karta.pdf...
Reading VIG07A_30.09.2022_Karta.pdf...
Reading VIG06_30.09.2022_Karta.pdf...
Reading VIG06A_30.09.

In [15]:
content = keep_kiids(content)

In [18]:
joblib.dump(content,"kiids.pkl")

['kiids.pkl']

In [11]:
META(content)

Saved to file: TMP_KIID_META.csv
